In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import pickle
from scipy.cluster import hierarchy as sch

In [ ]:
df = pd.read_csv("../../data/processed/clean_reviews.csv")
df.head()

In [ ]:
reviews = df['Cleaned Text'].tolist()
time = pd.to_datetime(df.Time).tolist()

In [ ]:
# Prepare embeddings
# sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# embeddings = sentence_model.encode(reviews, show_progress_bar=True)

In [ ]:
# save embeddings
# with open('reviews_BERT_embedding.pickle', 'wb') as pkl:
#     pickle.dump(embeddings, pkl)

In [ ]:
# load embeddings
with open('reviews_BERT_embedding.pickle', 'rb') as pkl:
    embeddings = pickle.load(pkl)

In [ ]:
topic_model = BERTopic(language="english", min_topic_size=20)
topics, probs = topic_model.fit_transform(reviews, embeddings)

In [ ]:
topic_model.get_topic_info()


In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
# Hierarchical topics
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(reviews, linkage_function=linkage_function)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [ ]:
topics_over_time = topic_model.topics_over_time(reviews, time, nr_bins=10)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10, normalize_frequency=True)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10, normalize_frequency=False)